In [ ]:
"""
LOAD THE DATA
"""
import os
import numpy as np
from parameters import FileParams as fp, DEFAULT_AUDIO_PARAMS as ap

def load_data(dataset: str):
    x = np.load(os.path.join(fp.data_dir, f'{dataset}_x.npy'))
    y = np.load(os.path.join(fp.data_dir, f'{dataset}_y.npy'))
    return x, y

train_x, train_y = load_data('train')
val_x, val_y = load_data('val')

print(f'Percent of train data positive: {np.mean(train_y) * 100:.2f}%')
print('train_x size: ', train_x.nbytes / 1024 / 1024, 'MB')
print(f'Percent of val data positive: {np.mean(val_y) * 100:.2f}%')
print('val_x size: ', val_x.nbytes / 1024 / 1024, 'MB')

In [ ]:
""" 
CREATE THE MODEL

Model architecture copied from
https://www.tensorflow.org/tutorials/audio/simple_audio
"""

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from keras.losses import binary_crossentropy

INPUT_SHAPE = (ap.n_features, ap.n_mfcc)
MODEL_NAME = 'simple_cnn'

# Instantiate the `tf.keras.layers.Normalization` layer.
norm_layer = layers.Normalization()
model = models.Sequential([
    layers.Input(shape=INPUT_SHAPE),
    norm_layer,
    layers.Conv1D(32, 3, activation='relu'),
    layers.Conv1D(64, 3, activation='relu'),
    layers.MaxPooling1D(),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid'),
], name=MODEL_NAME)

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=binary_crossentropy,
    metrics=['accuracy'],
)

In [ ]:
"""
SET UP FOR TRAINING
"""

from keras.callbacks import TensorBoard, ModelCheckpoint

BATCH_SIZE = 128
EPOCHS = 4

checkpoint_dir = f'./checkpoints/{MODEL_NAME}'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_dir,
    save_weights_only=False,  # Save the whole model
    monitor='val_accuracy',
    mode='auto',
    save_best_only=True)

log_dir = os.path.join(fp.log_dir, MODEL_NAME)
callbacks = [TensorBoard(log_dir), model_checkpoint_callback]
# Print out info about the model
model.summary()

In [ ]:
"""
TRAIN THE MODEL
"""
model.fit(
    train_x,
    train_y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(val_x, val_y),
    callbacks=callbacks
)

In [ ]:
"""
Evaluate on the validation set
"""

loss, accuracy = model.evaluate(val_x, val_y)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy*100:.5f}%')

In [ ]:
"""
USE THE MODEL TO MAKE PREDICTIONS
"""

predictions = model.predict(train_x[:10]).flatten()

for i in range(len(predictions)):
    print(f'Predicted: {predictions[i]:<.4f}, Actual: {train_y[i]}, {"Correct!" if round(predictions[i]) == train_y[i] else "Incorrect"}')

In [ ]:
"""
TEST SMALLER MODEL
"""

SMALLER_NAME = 'smaller_cnn'

# Same except half as many filters in the Conv1D layers
norm_layer = layers.Normalization()
smaller_model = models.Sequential([
    layers.Input(shape=INPUT_SHAPE),
    norm_layer,
    layers.Conv1D(16, 3, activation='relu'),
    layers.Conv1D(32, 3, activation='relu'),
    layers.MaxPooling1D(),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid'),
], name=SMALLER_NAME)

smaller_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.binary_crossentropy,
    metrics=['accuracy'],
)

smaller_checkpoint_dir = f'./checkpoints/{SMALLER_NAME}'
smaller_model_checkpoint_callback = ModelCheckpoint(
    filepath=smaller_checkpoint_dir,
    save_weights_only=False,  # Save the whole model
    monitor='val_accuracy',
    mode='auto',
    save_best_only=True)
smaller_callbacks = [
    TensorBoard(os.path.join(fp.log_dir, SMALLER_NAME)),
    model_checkpoint_callback
]
smaller_model.summary()

In [ ]:
"""
TRAIN THE SMALLER MODEL
"""
smaller_model.fit(
    train_x,
    train_y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(val_x, val_y),
    callbacks=smaller_callbacks
)